# POS-тэггинг с CRFSuite

CRFSuite — одна из нескольких систем для последовательного теггинга, основанных на CRF. В питоне есть обертка, `pycrfsuite`, которую мы и будем использовать.

В качестве тренировочного и тестового корпуса мы возьмем фрагмент НКРЯ в 1 млн словоупотреблений.

In [3]:
# Этот модуль нужно установить командой pip install python-crfsuite в консоли
import pycrfsuite
import codecs

# Это нужно заменить на папку с корпусом
%cd '~/Dropbox/Courses/HSE/CompLing/POS/'

/home/max/Dropbox/Courses/HSE/CompLing/POS


## 1 Загрузка корпуса

In [4]:
def read_ruscorpora(filename):
    sent = [[]]
    with codecs.open(filename) as inp_file:
        for line in inp_file:
            line = line.strip('\r\n ')
            if '\t' in line:
                line = line[:line.find('\t')]
            if line.startswith('#'):
                continue
            if not line:
                sent.append([])
                continue
            wordform, lemma, gram = line.rsplit('/', 2)
            pos = gram[:gram.find('=')] if '=' in gram else gram
            pos = pos.split(',')[0]
            sent[-1].append((wordform, pos, lemma, gram))
    
    return sent

In [5]:
corpus = read_ruscorpora('ruscorpora.parsed.txt')

In [6]:
for word in corpus[0]:
    print word[0], word[1], word[2], word[3]

Якутское A якутский A=n,sg,nom,plen
отделение S отделение S,n,inan=sg,nom
Единой A единый A=f,sg,gen,plen
России S Россия S,f,inan=sg,gen
планирует V планировать V,ipf,tran=sg,act,praes,3p,indic
создать V создать V,pf,tran=inf,act
комитеты S комитет S,m,inan=pl,acc
партийного A партийный A=m,sg,gen,plen
контроля S контроль S,m,inan=sg,gen
наподобие PR наподобие PR
существовавших V существовать V,ipf,intr,act=partcp,pl,gen,praet,plen
в PR в PR
СССР S СССР S,m,inan,0=sg,loc
на PR на PR
базе S база S,f,inan=sg,loc
КПСС S КПСС S,f,inan,0=sg,gen


## 2 Преобразование текста в фичи

Для каждого токена в качестве признаков мы будем использовать само слово и его контекст — несколько слов слева и справа. Количество контекста передается в аргументе `n_context`.

Если достаточного количества слов слева нет, то добавляется специальный признак начала предложения: `BOS`. Если нет контекста справа, то признак конца предложения: `EOS`.

In [7]:
def word2features(sent, i, n_context):
    word = sent[i][0]

    features = [
        'word=' + word
    ]
    
    for i_context in range(i-n_context, i):
        if i_context >= 0:
            features.extend([
                    '-{}:word='.format(i_context) + sent[i_context][0]
                ])
        else:
            features.append('BOS')
    for i_context in range(i+1, i+1+n_context):
        if i_context < len(sent):
            features.extend([
                    '+{}:word='.format(i_context) + sent[i_context][0]
                ])
        else:
            features.append('EOS')
    
    return features

Вспомогательные функции для преобразования предложения в последовательность признаков и тегов.

In [8]:
def sent2features(sent, n_context):
    return [word2features(sent, i, n_context) for i in range(len(sent))]

def sent2labels(sent):
    return [postag for token, postag, lemma, gram in sent]

def sent2tokens(sent):
    return [token for token, postag, lemma, gram in sent]

In [9]:
sent2features(corpus[0], 2)[0]

['word=\xd0\xaf\xd0\xba\xd1\x83\xd1\x82\xd1\x81\xd0\xba\xd0\xbe\xd0\xb5',
 'BOS',
 'BOS',
 '+1:word=\xd0\xbe\xd1\x82\xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5',
 '+2:word=\xd0\x95\xd0\xb4\xd0\xb8\xd0\xbd\xd0\xbe\xd0\xb9']

In [10]:
len(corpus)

90703

Разобьем корпус на тестовую и тренировочную выборку случайным образом

In [11]:
%%time
from sklearn import cross_validation

n_context = 2

X = [sent2features(s, n_context) for s in corpus]
y = [sent2labels(s) for s in corpus]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=0)
#X_train = [sent2features(s, n_context) for s in corpus[10000:]]
#y_train = [sent2labels(s) for s in corpus[10000:]]

#X_test = [sent2features(s, n_context) for s in corpus[:10000]]
#y_test = [sent2labels(s) for s in corpus[:10000]]

CPU times: user 4.71 s, sys: 255 ms, total: 4.97 s
Wall time: 9.05 s


Теперь можно тренировать CRF. Сначала создаем объект, потом добавляем ему тренировочный пары по одной.

In [12]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 5.66 s, sys: 101 ms, total: 5.76 s
Wall time: 5.58 s


## 3 Обучение
Зададим некоторые параметры обучения:

In [13]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [14]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

Запустим обучение:

In [15]:
%%time
trainer.train('ruscorpora.crfsuite')

CPU times: user 1min 30s, sys: 366 ms, total: 1min 30s
Wall time: 1min 30s


Теперь у нас есть сохраненная модель:

In [16]:
!ls -lh ./ruscorpora.crfsuite

-rw-r--r-- 1 max disk 5.4M Jan  1 14:58 ./ruscorpora.crfsuite


## 4 Загрузка натренированного тэггера и его тестирование

In [17]:
tagger = pycrfsuite.Tagger()
tagger.open('ruscorpora.crfsuite')

In [18]:
example_sent = corpus[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent, n_context))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Якутское отделение Единой России планирует создать комитеты партийного контроля наподобие существовавших в СССР на базе КПСС
('Predicted:', 'A S A S V V S A S PR S PR S PR S S')
('Correct:  ', 'A S A S V V S A S PR V PR S PR S S')


In [19]:
from sklearn.metrics import classification_report

In [27]:
y_pred = [tagger.tag(xseq) for xseq in X_test]
print classification_report(y_test, y_pred)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/multiclass.py:194: DeprecationWarning: Direct support for sequence of sequences multilabel representation will be unavailable from version 0.17. Use sklearn.preprocessing.MultiLabelBinarizer to convert to a label indicator representation.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/multiclass.py:194: DeprecationWarning: Direct support for sequence of sequences multilabel representation will be unavailable from version 0.17. Use sklearn.preprocessing.MultiLabelBinarizer to convert to a label indicator representation.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/multiclass.py:194: DeprecationWarning: Direct support for sequence of sequences multilabel representation will be unavailable from version 0.17. Use sklearn.preprocessing.MultiLabelBinarizer to convert to a label indicator representation.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/multi

             precision    recall  f1-score   support

          A       0.87      0.91      0.89     12261
      A-PRO       0.96      0.96      0.96      8868
        ADV       0.94      0.90      0.92      7958
    ADV-PRO       0.95      0.91      0.93      5340
       ANUM       0.92      0.73      0.81      1099
       CONJ       0.96      0.98      0.97     12397
       INIT       0.93      0.91      0.92       721
       INTJ       0.92      0.66      0.77       261
     NONLEX       0.94      0.62      0.75       887
        NUM       0.97      0.95      0.96      5423
    PARENTH       0.86      0.83      0.84      1332
       PART       0.97      0.94      0.95      8710
         PR       0.99      1.00      1.00     14316
    PRAEDIC       0.84      0.80      0.82      2084
PRAEDIC-PRO       0.86      0.86      0.86        21
          S       0.97      0.99      0.98     21145
      S-PRO       0.96      0.97      0.97     12373
          V       0.97      0.95      0.96   

/usr/local/lib/python2.7/dist-packages/sklearn/utils/multiclass.py:194: DeprecationWarning: Direct support for sequence of sequences multilabel representation will be unavailable from version 0.17. Use sklearn.preprocessing.MultiLabelBinarizer to convert to a label indicator representation.
  DeprecationWarning)


Мы можем заметить, что самое плохое качество достигается на наименее важных категориях: `NONLEX`, `INTJ` и `PARENTH`. То есть на важных грамматических категориях качество на самом деле выше.

Теперь попробуем разметить какое-нибудь предложение не из корпуса:

In [28]:
sent = u'Это тестовое предложение'
tagger.tag(sent2features([[item] for item in sent.split(' ')], 2))

['S-PRO', 'A', 'S']